# surprise library 설치

In [ ]:
conda install -c conda-forge scikit-surprise

Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yoonji\anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py38haa244fe_0         3.1 MB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    scikit-surprise-1.1.1      |   py38h1e00858_1         567 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/win-64::python_abi-3.8-1_cp38
  scikit-surprise    conda-forge/win-64::scikit-surprise-1.1.1-py38h1e00858_1

The following packages will be UPDATED:

  conda                       pkgs



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda




In [ ]:
# surprise library 업데이트
conda update -n base -c defaults conda

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\yoonji\anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    backports.functools_lru_cache-1.6.1|     pyhd3eb1b0_0          12 KB
    conda-4.9.2                |   py38haa95532_0         2.9 MB
    conda-package-handling-1.7.2|   py38h76e460a_0         723 KB
    ------------------------------------------------------------
                                           Total:         3.6 MB

The following packages will be UPDATED:

  conda-package-han~                   1.6.1-py38h62dcd97_0 --> 1.7.2-py38h76e460a_0

The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.9.2-py38haa244fe~ --> pkgs/main::conda-4.9.2-py38haa95532_0

The following packages will be DOWNGRADED:

  backports.functoo~   

In [ ]:
import surprise
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from surprise import Dataset, Reader

# 1. 데이터 로드

In [ ]:
ratings = pd.read_csv('C:/Users/yoonji/Desktop/kubig/추천 알고리즘 프로젝트/3주차-코딩 공부/item based/movie lens/ratings.csv')
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], 
                            reader=Reader(rating_scale=(0.5,5)))

In [ ]:
data.head()

AttributeError: 'DatasetAutoFolds' object has no attribute 'head'

# 2. 모델 검증을 위한 data split

In [ ]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  610 

Number of items:  8928 



# 3. 본격 알고리즘 구현

In [ ]:
from surprise import KNNWithMeans   

my_k = 15
my_min_k = 5

my_sim_option = {
    'name':'pearson', 'user_based':False, 
    }

algo = KNNWithMeans(
    k = my_k, min_k = my_min_k, sim_option = my_sim_option
    )

# 4. model fit & 평가

In [ ]:
algo.fit(trainset)

from surprise import accuracy
# testset에 적용 후 rmse 계산-작을수록 좋음
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9032


0.9032267875443455

In [ ]:
# cross validation을 통해 test set에 대한 overfitting 방지
from surprise.model_selection import cross_validate

results = cross_validate(
    algo = algo, data = data, measures=['RMSE'], 
    cv=5, return_train_measures=True
    )

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
results

{'test_rmse': array([0.89568342, 0.8977884 , 0.90334859, 0.89238532, 0.89158186]),
 'train_rmse': array([0.70759521, 0.70598756, 0.70566314, 0.70557797, 0.70878612]),
 'fit_time': (0.2443830966949463,
  0.2756221294403076,
  0.27597546577453613,
  0.2682209014892578,
  0.26329517364501953),
 'test_time': (1.569765567779541,
  1.5690727233886719,
  1.488018274307251,
  1.4787993431091309,
  1.4671239852905273)}

In [ ]:
# cross validation을 통해 구한 rmse들의 평균을 통해 모델 평가
results['test_rmse'].mean()

0.8961575193515425

In [ ]:
# 모델이 만족스러우면 full trainset에 fit-방금한 cross validation은 일부 trainset들에 적용한것이지 full train set 아님 
trainsetfull = data.build_full_trainset()
algo.fit(trainsetfull)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# full trainset에 fit시킨 모델을 testset에 적용
#test set=기존 데이터에서 rate 없는 부분  ex) 영화 100개 중 사용자가 15개만 평가했다면 나머지 85개가 test set
testset = trainsetfull.build_anti_testset()   
predictions = algo.test(testset)

In [ ]:
# predictions는 userid, itmid, 실제 평점, 예측 평점
predictions[:5]

[Prediction(uid=1, iid=318, r_ui=3.501556983616962, est=4.896401566324577, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=1, iid=1704, r_ui=3.501556983616962, est=4.447696742171961, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=1, iid=6874, r_ui=3.501556983616962, est=4.709470489962325, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=1, iid=8798, r_ui=3.501556983616962, est=4.397986210203815, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=1, iid=46970, r_ui=3.501556983616962, est=4.071055565085634, details={'actual_k': 15, 'was_impossible': False})]

In [ ]:
# 특정 사용자의 특정 아이템에 대한 선호도 예측
algo.predict(uid='1', iid='1')     #실제 평점이 없기 때문에 None

Prediction(uid='1', iid='1', r_ui=None, est=3.501556983616962, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

# top N 추천

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)      #dictionary 안을 list로 채우기
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))   #사용자 id 옆에 item id와 선호도 예측값 입력

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():   #items() : dictionary의 key와 value 함께 이용
        user_ratings.sort(key=lambda x: x[1], reverse=True)  #선호도 예측값을 내림차순 정렬
        top_n[uid] = user_ratings[:n]      #상위 n번째 영화 및 선호도 예측값까지 출력

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)   #사용자별로 10개 영화 추천

In [ ]:
top_n

defaultdict(list,
            {1: [(720, 5),
              (1266, 5),
              (2203, 5),
              (3365, 5),
              (3508, 5),
              (922, 5),
              (5902, 5),
              (1223, 5),
              (3022, 5),
              (3429, 5)],
             2: [(51931, 5),
              (27156, 5),
              (3266, 4.9934842093189005),
              (4021, 4.955992407928848),
              (3494, 4.897763507339918),
              (5747, 4.888432185752449),
              (177593, 4.884665659730955),
              (31364, 4.880690144386316),
              (1411, 4.858423533198317),
              (1279, 4.850988472796645)],
             3: [(106642, 3.780586122399865),
              (2925, 3.694507362142683),
              (2351, 3.6722271291076067),
              (2239, 3.653046705892477),
              (158966, 3.6450183917786827),
              (51931, 3.6409373148981095),
              (89904, 3.6140594734135743),
              (1964, 3.5915176079779063),


In [ ]:
# 사용자별 top_10 list
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])    #top_n에서 user id와 item id들만 추출

1 [720, 1266, 2203, 3365, 3508, 922, 5902, 1223, 3022, 3429]
2 [51931, 27156, 3266, 4021, 3494, 5747, 177593, 31364, 1411, 1279]
3 [106642, 2925, 2351, 2239, 158966, 51931, 89904, 1964, 86377, 1197]
4 [27156, 51931, 177593, 1276, 6460, 158966, 3451, 3266, 1247, 1223]
5 [1204, 1201, 912, 720, 51931, 6460, 106642, 3266, 177593, 1250]
6 [51931, 6460, 3451, 720, 2732, 1617, 6666, 106642, 2202, 6791]
7 [27156, 51931, 177593, 6460, 3451, 106642, 2239, 3266, 6666, 2202]
8 [1204, 720, 177593, 51931, 6460, 3451, 2732, 5747, 1387, 106642]
9 [2239, 3451, 5747, 3494, 6460, 6666, 3266, 1732, 1683, 106642]
10 [27156, 51931, 3451, 1683, 106642, 177593, 904, 158966, 720, 3275]
11 [720, 3451, 6460, 7008, 3201, 26133, 2239, 2202, 2732, 177593]
12 [50, 457, 593, 1617, 2078, 2395, 318, 77455, 86345, 720]
13 [27156, 2239, 6460, 3451, 51931, 26133, 1223, 2202, 170705, 106642]
14 [3266, 6460, 51931, 2239, 177593, 6666, 3451, 2732, 106642, 720]
15 [27156, 177593, 51931, 1223, 6460, 48516, 3451, 720, 3266, 220

In [ ]:
# 사용자 1명에 대한 추천 list
movies = pd.read_csv('C:/Users/yoonji/Desktop/kubig/추천 알고리즘 프로젝트/3주차-코딩 공부/item based/movie lens/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
def user_recommendation(userId) :
    recommend_movie_title = []
    for i in range(10) : recommend_movie_title.append(top_n[userId][i][0])
    recommend_movie_title = np.array(recommend_movie_title, dtype=int)

    recommend_movies = movies[movies['movieId'].isin(recommend_movie_title)]

    recommend_movie_rating = []
    for i in range(10) : recommend_movie_rating.append(top_n[userId][i][1])
    recommend_movie_rating = np.array(recommend_movie_rating)

    recommend_movie_rating = pd.DataFrame({'movieId' : recommend_movie_title, 
                                       'rating_est' : recommend_movie_rating})


    print("The Recommendations for User Id : ",userId)
    print(' ')
    print('Top-10 movie recommendation lists')
    return pd.merge(recommend_movie_rating, recommend_movies, on='movieId')

In [ ]:
user_recommendation(1)

The Recommendations for User Id :  1
 
Top-10 movie recommendation lists


,movieId,rating_est,title,genres
0,720,5,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy
1,1266,5,Unforgiven (1992),Drama|Western
2,2203,5,Shadow of a Doubt (1943),Crime|Drama|Thriller
3,3365,5,"Searchers, The (1956)",Drama|Western
4,3508,5,"Outlaw Josey Wales, The (1976)",Action|Adventure|Drama|Thriller|Western
5,922,5,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Drama|Film-Noir|Romance
6,5902,5,Adaptation (2002),Comedy|Drama|Romance
7,1223,5,"Grand Day Out with Wallace and Gromit, A (1989)",Adventure|Animation|Children|Comedy|Sci-Fi
8,3022,5,"General, The (1926)",Comedy|War
9,3429,5,Creature Comforts (1989),Animation|Comedy


참고) 상세 설명  ex) 1번에게 추천

In [ ]:
recommend_movie_title = []
for i in range(10) : recommend_movie_title.append(top_n[1][i][0])
recommend_movie_title = np.array(recommend_movie_title, dtype=int)
recommend_movie_title

array([ 720, 1266, 2203, 3365, 3508,  922, 5902, 1223, 3022, 3429])

In [ ]:
recommend_movies = movies[movies['movieId'].isin(recommend_movie_title)]
recommend_movies

,movieId,title,genres
585,720,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy
704,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Drama|Film-Noir|Romance
924,1223,"Grand Day Out with Wallace and Gromit, A (1989)",Adventure|Animation|Children|Comedy|Sci-Fi
965,1266,Unforgiven (1992),Drama|Western
1650,2203,Shadow of a Doubt (1943),Crime|Drama|Thriller
2279,3022,"General, The (1926)",Comedy|War
2517,3365,"Searchers, The (1956)",Drama|Western
2562,3429,Creature Comforts (1989),Animation|Comedy
2623,3508,"Outlaw Josey Wales, The (1976)",Action|Adventure|Drama|Thriller|Western
4117,5902,Adaptation (2002),Comedy|Drama|Romance


In [ ]:
recommend_movie_rating = []
for i in range(10) : recommend_movie_rating.append(top_n[1][i][1])
recommend_movie_rating = np.array(recommend_movie_rating)
recommend_movie_rating

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [ ]:
recommend_movie_rating = pd.DataFrame({'movieId' : recommend_movie_title, 
                                       'rating_est' : recommend_movie_rating})
recommend_movie_rating

,movieId,rating_est
0,720,5
1,1266,5
2,2203,5
3,3365,5
4,3508,5
5,922,5
6,5902,5
7,1223,5
8,3022,5
9,3429,5


In [ ]:
pd.merge(recommend_movie_rating, recommend_movies, on='movieId')

,movieId,rating_est,title,genres
0,720,5,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy
1,1266,5,Unforgiven (1992),Drama|Western
2,2203,5,Shadow of a Doubt (1943),Crime|Drama|Thriller
3,3365,5,"Searchers, The (1956)",Drama|Western
4,3508,5,"Outlaw Josey Wales, The (1976)",Action|Adventure|Drama|Thriller|Western
5,922,5,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Drama|Film-Noir|Romance
6,5902,5,Adaptation (2002),Comedy|Drama|Romance
7,1223,5,"Grand Day Out with Wallace and Gromit, A (1989)",Adventure|Animation|Children|Comedy|Sci-Fi
8,3022,5,"General, The (1926)",Comedy|War
9,3429,5,Creature Comforts (1989),Animation|Comedy


# 모델들 비교

In [ ]:
# 다양한 유사도 함수
sim_msd = {'name':'MSD', 'user_based':False}
sim_cos = {'name':'cosine', 'user_based':False}
sim_pearson = {'name':'pearson', 'user_based':False}
sim_pearson_baseline = {'name': 'pearson_baseline','user_based':False, 'shrinkage': 100}

sim_options = [sim_msd, sim_cos, sim_pearson, sim_pearson_baseline]

In [ ]:
my_k = 15
my_min_k = 5

test_rmse = []

# KNNBasic
for curr_sim_option in sim_options :
    algo = surprise.KNNBasic(k = my_k, min_k = my_min_k, sim_options = curr_sim_option)
    results = cross_validate(algo = algo, data = data, measures=['RMSE'], 
                             cv=3, return_train_measures=True)
    test_rmse.append(results['test_rmse'].mean())
    
test_rmse    

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done 

[0.9332181694387162,
 1.0030154106687874,
 0.9927854514216111,
 0.9347425035177955]

In [ ]:
#KNN with Means
for curr_sim_option in sim_options :
    algo = surprise.KNNWithMeans(k = my_k, min_k = my_min_k, sim_options = curr_sim_option)
    results = cross_validate(algo = algo, data = data, measures=['RMSE'], 
                             cv=3, return_train_measures=True)
    test_rmse.append(results['test_rmse'].mean())
    
test_rmse  

In [ ]:
# KNN with Zscore
for curr_sim_option in sim_options :
    algo = surprise.KNNWithZScore(k = my_k, min_k = my_min_k, sim_options = curr_sim_option)
    results = cross_validate(algo = algo, data = data, measures=['RMSE'], 
                             cv=3, return_train_measures=True)
    test_rmse.append(results['test_rmse'].mean())
    
test_rmse  

In [ ]:
# KNN baseline
for curr_sim_option in sim_options :
    algo = surprise.KNNBaseline(k = my_k, min_k = my_min_k, sim_options = curr_sim_option)
    results = cross_validate(algo = algo, data = data, measures=['RMSE'], 
                             cv=3, return_train_measures=True)
    test_rmse.append(results['test_rmse'].mean())
    
test_rmse  

In [ ]:
# SVD
algo = surprise.SVD()
results = cross_validate(algo = algo, data = data, measures=['RMSE'], 
                             cv=3, return_train_measures=True)
test_rmse.append(results['test_rmse'].mean())
    
test_rmse  

[0.9332181694387162,
 1.0030154106687874,
 0.9927854514216111,
 0.9347425035177955,
 0.8794431137034501]

In [ ]:
# SVD++
algo = surprise.SVDpp()
results = cross_validate(algo = algo, data = data, measures=['RMSE'], 
                             cv=3, return_train_measures=True)
test_rmse.append(results['test_rmse'].mean())
    
test_rmse  

[0.9332181694387162,
 1.0030154106687874,
 0.9927854514216111,
 0.9347425035177955,
 0.8794431137034501,
 0.8684859551613872]

데이터프레임으로 깔끔하게 결과 보기

In [ ]:
algorithm = np.array(['KNN Basic','KNN with Means','KNN with Zscore','KNN baseline'])
algorithm = np.r_[np.repeat(algorithm, 4), np.array(['SVD','SVD++'])]
algorithm

array(['KNN Basic', 'KNN Basic', 'KNN Basic', 'KNN Basic',
       'KNN with Means', 'KNN with Means', 'KNN with Means',
       'KNN with Means', 'KNN with Zscore', 'KNN with Zscore',
       'KNN with Zscore', 'KNN with Zscore', 'KNN baseline',
       'KNN baseline', 'KNN baseline', 'KNN baseline', 'SVD', 'SVD++'],
      dtype='<U15')

In [ ]:
sim_option = ['msd','cos','pearson','pearson_baseline']*4 + ['NA', 'NA']
sim_option

['msd',
 'cos',
 'pearson',
 'pearson_baseline',
 'msd',
 'cos',
 'pearson',
 'pearson_baseline',
 'msd',
 'cos',
 'pearson',
 'pearson_baseline',
 'msd',
 'cos',
 'pearson',
 'pearson_baseline',
 'NA',
 'NA']

In [ ]:
compare_models = pd.DataFrame({'algorithm' : algorithm,'sim_option' : sim_option,'test_rmse':test_rmse})
compare_models

,algorithm,sim_option
0,KNN Basic,msd
1,KNN Basic,cos
2,KNN Basic,pearson
3,KNN Basic,pearson_baseline
4,KNN with Means,msd
5,KNN with Means,cos
6,KNN with Means,pearson
7,KNN with Means,pearson_baseline
8,KNN with Zscore,msd
9,KNN with Zscore,cos


예상대로 SVD++이 가장 낮은 rmse